In [ ]:
from inner_grid import GridGenerator
from shapely.geometry import Polygon
import geopandas as gpd
from shapely import wkt
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import pandas as pd
from tqdm import tqdm
import jaydebeapi as jdbc
import jpype
import os
from prepareData import prepare_data
from convert import convert_polygon

grid = GridGenerator()
#!!!!!!!define polygon(s) to prepare in geojson format
file = r""
path = r""
#!!!!!!!define type of data formal/informal/real - infromal from shelter associates are cut and prepared 
classification = "formal"
#50m squares all informal settlements.json
#100m squares all informal settlements.json
#!!!!!!!define cut size 50mx50m or 100mx100m in int format 50/100 (in case of different size, values in next steps (lat,lon) needs to be changed/added)
cut_size = 50


In [ ]:

#read the choosen area to cut into squares in geojson format
polygons = gpd.read_file(f"{path}\{file}")
all_grids = []

<string>:2: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
<string>:2: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
C:\Users\JulianaBaumgartnerov\AppData\Local\Temp\ipykernel_38244\2380777913.py:2: SyntaxWarning: invalid escape sequence '\{'
  polygons = gpd.read_file(f"{path}\{file}")


In [ ]:
#cutting into squares - size defined in 1st tile
total = len(polygons)

for idx, polygon in enumerate(polygons["geometry"]):
    progress = f"{idx + 1}/{total}"
    print(f"Processing polygon {progress}...")

    if cut_size == 50:
        inside_grid = grid.rectangles_inside_polygon(polygon=polygon, size=(0.000451369, 0.00045121))
    elif cut_size == 100:
        inside_grid = grid.rectangles_inside_polygon(polygon=polygon, size=(0.000902738, 0.00090242))

    all_grids.extend(list(inside_grid))

gdf = gpd.GeoDataFrame(all_grids, columns=["geometry"])
cut_file = gdf.to_file(f'{file}_{classification}_cut_{cut_size}m.json', driver='GeoJSON')


Processing polygon 1/1...


c:\Users\JulianaBaumgartnerov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [ ]:
#!!!!!!change name of table accordingly
sql_tablename = "FEATURES_DB_MAHARASHTRA"

# connect to the IBM DB2 function
def connect_to_db():

    jar = "db2jcc4.jar"
    # os.environ['PATH'] = 'C:\Program Files\Java\jdk-22.0.1\\bin:' + os.environ['PATH']
    os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-17\\bin\server"
    os.environ["CLASSPATH"] = jar
    # print(os.environ)

    args = "-Djava.class.path=%s" % jar
    jvm_path = jpype.getDefaultJVMPath()
    try:
        jpype.startJVM(jvm_path, args, convertStrings=True)
    except Exception as e:
        # print('startJVM exception: ', e)
        pass

    if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():
        jpype.attachThreadToJVM()
        jpype.java.lang.Thread.currentThread().setContextClassLoader(
            jpype.java.lang.ClassLoader.getSystemClassLoader()
        )

    username = "xxxxxx"
    password = "xxxxxx"

    drvrpth = "db2jcc4.jar"
    drvrnm = "com.ibm.db2.jcc.DB2Driver"

    connstr = "jdbc:db2://65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371/BLUDB:sslConnection=true;useJDBC4ColumnNameAndLabelSemantics=false;db2.jcc.charsetDecoderEncoder=3;"

    dw_active = "yes"
    dm_active = "yes"

    host = "65beb513-5d3d-4101-9001-f42e9dc954b3.brt9d04f0cmqeb8u7740.databases.appdomain.cloud:30371"
    jdbc_driver_name = "com.ibm.db2.jcc.DB2Driver"
    #driver needs to be in same folder
    jdbc_driver_loc = "db2jcc4.jar"

    dw_active = "yes"
    dm_active = "yes"

    conn = jdbc.connect(drvrnm, connstr, [username, password], drvrpth)

    return conn


def fetch_builings(cursor, max_lon, min_lon, min_lat, max_lat):
    """
    This particular function is aimed for obtating all entries from defined rectangle for selected SQL table
    """

    sql = f"""
        SELECT
            count(ID) as count,
            avg(area_in_meters) as avg_area,
            max(area_in_meters) as max_area,
            avg(height) as avg_height
        FROM USER1.{sql_tablename}
        WHERE
            LONGITUDE <= {max_lon} AND 
            LONGITUDE >= {min_lon} AND 
            LATITUDE >= {min_lat} AND 
            LATITUDE <= {max_lat}
    """

    try:
        cursor.execute(sql)
        data = cursor.fetchall()
    except Exception as e:
        print(f"Fetch items error occurred: {e}")
        print("Reconnecting to the database try again...")

        conn = connect_to_db()
        cursor = conn.cursor()
        cursor.execute(sql)
        data = cursor.fetchall()
    finally:
        # reshape obtained data to the GeoDataFrame
        # df = pd.DataFrame(data=data, columns=columns)
        # df = gpd.GeoDataFrame(
        #     df, geometry=shapely.from_wkt(df.polygon_coordinates.astype(str))
        # )
        # df = df.drop(["polygon_coordinates"], axis=1)
        return data


if __name__ == "__main__":
    file = (f'{file}_{classification}_cut_{cut_size}m.json')
    #gdf_roads = gpd.read_file("/real_with_roads.geojson")  
    gdf_bounds = prepare_data(file)
    rows = []

    conn = connect_to_db()
    cursor = conn.cursor()

    chunk_size = 200
    rows = []
    total_chunks = (len(gdf_bounds) + chunk_size - 1) // chunk_size

    for chunk_index, chunk_start in enumerate(range(0, len(gdf_bounds), chunk_size), start=1):
        chunk_end = min(chunk_start + chunk_size, len(gdf_bounds))
        chunk = gdf_bounds.iloc[chunk_start:chunk_end]

        conn = connect_to_db()
        cursor = conn.cursor()
        print(f"Processing chunk {chunk_index} of {total_chunks}...")

        for idx, row in chunk.iterrows():
            id = row.id
            polygon = row.geometry
            max_lon = row.max_lon
            min_lon = row.min_lon
            min_lat = row.min_lat
            max_lat = row.max_lat

            # Get data from database
            data_from_db = fetch_builings(cursor, max_lon, min_lon, min_lat, max_lat)
            #parameters from DB
            count, avg_area, max_area, avg_height = data_from_db[0]

            # roads count 
            """
            roads_in_polygon = gdf_roads[gdf_roads.intersects(polygon)]
            if len(roads_in_polygon) == 0:
                is_road = 0
            else:
                is_road = 1
            """ 

            #, res_count, non_res_count

            # # Convert polygon to GeoJSON
            converted_polygon = convert_polygon(polygon=polygon)

            # Create row with parameters from db
            db_row = {
            "id": id,
            "class": classification,
            "geometry": converted_polygon,
            "polygon": polygon,
            "count": count,
            "avg_area": avg_area,
            # "median_area": median_area,
            "max_area": max_area,
            "avg_height": avg_height,
            #"res_count":res_count,
           #"non_res_count":non_res_count,
        }

            rows.append(db_row)


        cursor.close()
        conn.close()

        print(f"Finished processing chunk {chunk_index} of {total_chunks}.\n")


df = pd.DataFrame(data=rows)


#df.to_csv(f"{file}_buildings.csv")
#print(df)

<>:9: SyntaxWarning: invalid escape sequence '\P'
<>:9: SyntaxWarning: invalid escape sequence '\P'
C:\Users\JulianaBaumgartnerov\AppData\Local\Temp\ipykernel_38244\386773959.py:9: SyntaxWarning: invalid escape sequence '\P'
  os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-17\\bin\server"
C:\Users\JulianaBaumgartnerov\AppData\Local\Temp\ipykernel_38244\386773959.py:21: DeprecationWarning: jpype._core.isThreadAttachedToJVM is deprecated, use java.lang.Thread.isAttached instead
  if jpype.isJVMStarted() and not jpype.isThreadAttachedToJVM():


Processing chunk 1 of 6...
Finished processing chunk 1 of 6.

Processing chunk 2 of 6...
Finished processing chunk 2 of 6.

Processing chunk 3 of 6...
Finished processing chunk 3 of 6.

Processing chunk 4 of 6...
Finished processing chunk 4 of 6.

Processing chunk 5 of 6...
Finished processing chunk 5 of 6.

Processing chunk 6 of 6...
Finished processing chunk 6 of 6.



In [ ]:
print(df.head())

                                     id   class  \
0   72.88946191812047:19.05055549589045  formal   
1   72.88946191812047:19.05100670589045  formal   
2  72.88946191812047:19.051457915890452  formal   
3  72.88946191812047:19.051909125890454  formal   
4  72.88946191812047:19.052360335890455  formal   

                                            geometry  \
0  {"type": "Feature", "geometry": {"type": "Poly...   
1  {"type": "Feature", "geometry": {"type": "Poly...   
2  {"type": "Feature", "geometry": {"type": "Poly...   
3  {"type": "Feature", "geometry": {"type": "Poly...   
4  {"type": "Feature", "geometry": {"type": "Poly...   

                                             polygon  count    avg_area  \
0  POLYGON ((72.88991328712048 19.05055549589045,...      4  122.332600   
1  POLYGON ((72.88991328712048 19.05100670589045,...      3  142.202667   
2  POLYGON ((72.88991328712048 19.051457915890452...      2  168.976200   
3  POLYGON ((72.88991328712048 19.051909125890454...    

In [7]:
squares = gpd.GeoDataFrame(df, geometry="polygon", crs="EPSG:4326")

In [ ]:
# check number of rows
print(squares["id"].count())

In [8]:
tiles = gpd.read_file("tiles.geojson")
tiles = tiles.to_crs(squares.crs)

# Spatial join: which tile intersects which square
join = gpd.sjoin(squares, tiles, how="left", predicate="intersects")

##### Check if there are any duplicates
- rows are duplicated in case our 50x50m large square intersect with more tiles

In [ ]:
print(f'Number of rows: {join["id"].count()}')
duplicates_in_id = join[join.duplicated("id", keep=False)]

print(f'Number of duplicates in column "id": {duplicates_in_id["id"].count()} \n')
print("Duplicates in column 'id':")
print(duplicates_in_id["id"].head())

##### Download required tiffs

In [9]:
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))
os.makedirs("india_tiles", exist_ok=True)

tile_ids = join["tile"].dropna().unique()

for tile_id in tile_ids:
    tif_path = f"india_tiles/{tile_id}.tif"
    tile_key = f"forests/v1/alsgedi_global_v6_float/chm/{tile_id}.tif"
    if not os.path.exists(tif_path):
        try:
            s3.download_file("dataforgood-fb-data", tile_key, tif_path)
            print(f"Downloaded {tile_id}.tif")
        except Exception as e:
            print(f"Failed to download {tile_id}.tif: {e}")

##### Count the number of pixels in the square
- currently hardcoded: over 3m

In [10]:
heights = []

for idx, row in tqdm(join.iterrows(), total=len(join)):
    geom = row.polygon
    tile_id = row["tile"]
    tif_path = f"india_tiles/{tile_id}.tif"

    try:
        with rasterio.open(tif_path) as src:
            # Reproject square to raster CRS
            square = gpd.GeoSeries([geom], crs=squares.crs).to_crs(src.crs).iloc[0]

            # Clip and extract
            out_image, _ = mask(src, [mapping(square)], crop=True)
            data = out_image[0]
            valid = data[data != src.nodata]

            # Collect stats
            if valid.size > 0:
                count_ge_3m = (valid >= 3).sum()

                heights.append(
                    {
                        "id": row.name,
                        "tile": tile_id,
                        "num_pixels_ge_3m": int(count_ge_3m),
                    }
                )
            else:
                heights.append(
                    {
                        "id": row.name,
                        "tile": tile_id,
                        "num_pixels_ge_3m": int(count_ge_3m),
                    }
                )

    except Exception as e:
        print(f"Error with tile {tile_id} and square {row.name}: {e}")

# Convert to DataFrame
df_heights = pd.DataFrame(heights)

100%|██████████| 1156/1156 [00:16<00:00, 70.17it/s]


##### Merge duplicated rows together, on "id"
- "id" is row.name (not longitude:latitude)
- tile names are concatenated
- the pixel counts from multiple rows are added together

In [ ]:
sum_multiple_tiles = df_heights.groupby("id", as_index=False).agg(
    {"tile": lambda x: " | ".join(x.unique()), "num_pixels_ge_3m": "sum"}
)

# sum_multiple_tiles.columns
# print(sum_multiple_tiles[sum_multiple_tiles["id"] == 14684])
# print(f"count: {sum_multiple_tiles['id'].count()}")

In [ ]:
# Ensure 'id' is the index in df_heights
sum_multiple_tiles = sum_multiple_tiles.set_index("id")

# Join with original GeoDataFrame
squares_with_heights = df.join(sum_multiple_tiles, how="left")

In [ ]:
squares_with_heights.to_csv(
    r"formal_cut_50m.json_buildings_trees.csv"
)